<a href="https://colab.research.google.com/github/kumariAkanksha666/kumari-Akanksha/blob/main/Copy_of_food_analysis_project_akanksha_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:





# 0. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change working directory to a folder in your Drive so files are saved permanently
import os
os.makedirs('/content/drive/MyDrive/food_waste_project', exist_ok=True)
os.chdir('/content/drive/MyDrive/food_waste_project')
# 1. Import libraries
import sqlite3
import pandas as pd

# 2. Load CSV files
providers_df = pd.read_csv("providers_data.csv")
receivers_df = pd.read_csv("receivers_data.csv")
food_listings_df = pd.read_csv("food_listings_data.csv")
claims_df = pd.read_csv("claims_data.csv")

# -------- Data Cleaning --------
# Remove duplicates
providers_df.drop_duplicates(inplace=True)
receivers_df.drop_duplicates(inplace=True)
food_listings_df.drop_duplicates(inplace=True)
claims_df.drop_duplicates(inplace=True)

# Remove leading/trailing spaces in string columns
for df in [providers_df, receivers_df, food_listings_df, claims_df]:
    df.columns = df.columns.str.strip()  # clean column names
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()

# Handle missing values
providers_df.fillna({"Name": "Unknown", "City": "Unknown", "Contact": "Unknown"}, inplace=True)
receivers_df.fillna({"Name": "Unknown", "City": "Unknown", "Contact": "Unknown"}, inplace=True)
food_listings_df.fillna({"Food_Name": "Unknown", "Location": "Unknown", "Food_Type": "Unknown"}, inplace=True)
claims_df.fillna({"Status": "Unknown"}, inplace=True)

# Ensure date format is consistent (YYYY-MM-DD)
food_listings_df['Expiry_Date'] = pd.to_datetime(food_listings_df['Expiry_Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# 3. Create SQLite DB and store cleaned tables
conn = sqlite3.connect("food_waste.db")
providers_df.to_sql("providers", conn, if_exists="replace", index=False)
receivers_df.to_sql("receivers", conn, if_exists="replace", index=False)
food_listings_df.to_sql("food_listings", conn, if_exists="replace", index=False)
claims_df.to_sql("claims", conn, if_exists="replace", index=False)

# 4. 15 SQL Queries
q1 = pd.read_sql_query("""
SELECT p.City,
COUNT(DISTINCT p.Provider_ID) AS Providers_Count,
COUNT(DISTINCT r.Receiver_ID) AS Receivers_Count
FROM providers p
LEFT JOIN receivers r ON p.City = r.City
GROUP BY p.City
""", conn)

q2 = pd.read_sql_query("""
SELECT Provider_Type,
COUNT(*) AS Total_Listings
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Listings DESC
""", conn)

q3 = pd.read_sql_query("""
SELECT Name, Contact
FROM providers
WHERE City = 'Chennai'
""", conn)

q4 = pd.read_sql_query("""
SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Name
ORDER BY Total_Claims DESC
""", conn)

q5 = pd.read_sql_query("""
SELECT SUM(Quantity) AS Total_Quantity_Available
FROM food_listings
""", conn)

q6 = pd.read_sql_query("""
SELECT Location AS City, COUNT(*) AS Listings_Count
FROM food_listings
GROUP BY Location
ORDER BY Listings_Count DESC
""", conn)

q7 = pd.read_sql_query("""
SELECT Food_Type, COUNT(*) AS Count
FROM food_listings
GROUP BY Food_Type
ORDER BY Count DESC
""", conn)

q8 = pd.read_sql_query("""
SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claims_Count
FROM food_listings f
LEFT JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Food_Name
ORDER BY Claims_Count DESC
""", conn)

q9 = pd.read_sql_query("""
SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
""", conn)

q10 = pd.read_sql_query("""
SELECT Status,
ROUND((COUNT() * 100.0 / (SELECT COUNT() FROM claims)), 2) AS Percentage
FROM claims
GROUP BY Status
""", conn)

q11 = pd.read_sql_query("""
SELECT r.Name, ROUND(AVG(f.Quantity), 2) AS Avg_Quantity_Claimed
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Name
""", conn)

q12 = pd.read_sql_query("""
SELECT Meal_Type, COUNT(*) AS Claims_Count
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY Meal_Type
ORDER BY Claims_Count DESC
""", conn)

q13 = pd.read_sql_query("""
SELECT p.Name, SUM(f.Quantity) AS Total_Donated
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC
""", conn)

q14 = pd.read_sql_query("""
SELECT Food_Name, Expiry_Date
FROM food_listings
WHERE DATE(Expiry_Date) <= DATE('now', '+2 days')
""", conn)

q15 = pd.read_sql_query("""
SELECT f.Location AS City,
COUNT(DISTINCT f.Food_ID) AS Donations,
COUNT(DISTINCT c.Claim_ID) AS Claims
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Location
ORDER BY Donations DESC, Claims DESC
""", conn)

# 5. Save database and close connection
conn.commit()
conn.close()

# 6. Download the SQLite database to your computer
from google.colab import files
files.download("food_waste.db")

Mounted at /content/drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>